# 1. 基础设置

In [3]:
# 导包
import torch
import os
import argparse
import pandas as pd
import tqdm
import numpy as np1
# os.environ['NUMEXPR_MAX_THREADS'] = '48'

# 导入各个指标
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
from ppsplit.quantification.distance_correlation.distCor import distCorMetric
from ppsplit.quantification.fisher_information.dFIL_inverse import dFILInverseMetric
from ppsplit.quantification.shannon_information.mutual_information import MuInfoMetric
from ppsplit.quantification.shannon_information.ULoss import ULossMetric
from ppsplit.quantification.rep_reading.rep_reader import PCA_Reader
from ppsplit.quantification.shannon_information.ITE_tools import Shannon_quantity

from target_model.task_select import get_dataloader_and_model

# utils
from ppsplit.utils.utils import create_dir

In [4]:
args = {
        'device':torch.device("cuda:1" if torch.cuda.is_available() else "cpu"),
        # 'device':torch.device("cpu"),
        # 'dataset':'CIFAR10',
        # 'dataset':'bank',
        # 'dataset':'credit',
        # 'dataset':'purchase',
        'dataset':'Iris',
        'result_dir': '20240702-FIL/',
        'oneData_bs':30,
        'noise_scale':0, # 防护措施
        'OneData':True,
        'split_layer': 1,
        }
print(args['device'])

cuda:1


In [5]:
# 获取数据集和模型
# if OneData:
#     return one_data_loader,client_net,results_dir,decoder_route
# else:
#     return trainloader,testloader,client_net,results_dir,decoder_route

# one_data_loader,client_net,results_dir,decoder_route = get_dataloader_and_model(dataset=args['dataset'], 
#                                                                                 loader_bs=1, 
#                                                                                 oneData_bs=args['batch_size'], 
#                                                                                 noise_scale=args['noise_scale'], 
#                                                                                 result_ws=args['result_dir'], 
#                                                                                 OneData=True)

one_data_loader,client_net,results_dir,decoder_route = get_dataloader_and_model(**args)
# client_net.to(args['device'])
# create_dir(results_dir)
print('results_dir:',results_dir)

===============processing data===============
X_train.shape: (120, 4)
X_test.shape: (30, 4)
y_train.shape: (120,)
y_test.shape: (30,) <class 'numpy.ndarray'>
===============processing data end===============


linear1.weight
linear1.bias
results_dir: ../results/20240702-FIL//Iris/1/


# 2. pytorch的自动梯度

In [6]:
# 前向传播
import torch

x = torch.ones(5, requires_grad=True)  # input tensor
y = torch.zeros(3, requires_grad=True)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

# 信息打印
print(x.shape)
print(x.requires_grad)
print(z.shape)

torch.Size([5])
True
torch.Size([3])


In [ ]:
# 反向传播
loss.backward()
print(x.grad.shape)

In [ ]:
# grad/backward 张量求梯度 sum
x = torch.ones(5, requires_grad=True)  # input tensor
y = torch.zeros(3, requires_grad=True)  # expected output
w = torch.randn(5, 3)
b = torch.randn(3)
z = torch.matmul(x, w)+b

# 计算sumed雅可比矩阵
z.backward(torch.ones_like(z))
print(x.grad.shape)
print(x.grad)

# 用grad
from torch.autograd import grad
x = torch.ones(5, requires_grad=True)  # input tensor
y = torch.zeros(3, requires_grad=True)  # expected output
w = torch.eye(5, 3)
b = torch.randn(3)
z = torch.matmul(x, w)+b
xgrad = grad(z, x, grad_outputs=torch.ones_like(z))[0]
print(xgrad)


In [ ]:
# jacobian
from torch.autograd.functional import jacobian
from torch.autograd import grad
x = torch.ones(5, requires_grad=True)  # input tensor

def forward(x):
    y = torch.zeros(3, requires_grad=True)  # expected output
    w = torch.eye(5, 3)
    b = torch.randn(3)
    z = torch.matmul(x, w)+b
    return z


# 计算sumed雅可比矩阵
jac = jacobian(forward, x)
print(jac.shape)
print(jac)
print(jac.sum(dim=0))

# 3. 试探FIL计算

In [ ]:
# FIL计算：
import torch.autograd.functional as F
# 切割模型通讯量查看
# for i in range(7):
vgg5 = VGG('Client', 'VGG5', 1, model_cfg)

client_outputs = vgg5(images)
print('outputs.shape:',client_outputs.shape)
jacs = F.jacobian(vgg5, images)
print('jacobian: ', jacs)
# print('output size:',torch.prod(torch.tensor(list(client_outputs.shape))[1:]))
print('output size:',torch.prod(torch.tensor(list(client_outputs.shape))))

# 0到6层每层的jacobians
import torch.autograd.functional as F
for i in range(7):
    vgg5 = VGG('Client', 'VGG5', i, model_cfg)
    client_outputs = vgg5(images)
    print('outputs.shape:',client_outputs.shape)
    jacs = F.jacobian(vgg5, images)
    print('jacobian: ', jacs)
    # print('output size:',torch.prod(torch.tensor(list(client_outputs.shape))[1:]))
    print('output size:',torch.prod(torch.tensor(list(client_outputs.shape))))



In [ ]:
# FIL 计算，摸索出来一条路
import torch.autograd.functional as F
# 参数：
sigma = 0.01

# 计算jacobian
# 取一个batch的数据
train_iter=iter(trainloader)
inputs,labels = train_iter.next()6
print("inputs.shape: ",inputs.shape)
print("labels.shape: ",labels.shape)
print(f"input.requires_grad: {inputs.requires_grad}")

# 加载模型：
# vgg5 = VGG('Client', 'VGG5', 1, model_cfg)

# 进行前向传播：
inputs.requires_grad_(True) # 需要求导
outputs = vgg5(inputs)
outputs = outputs + sigma * torch.randn_like(outputs) # 加噪声 (0,1] uniform
print("outputs.shape: ",outputs.shape)

# 1. 进行反向传播,计算jacobian
# outputs.backward(torch.ones_like(outputs))
# J = inputs.grad / sigma # 计算jacobian
# print(f"J1.shape: {J.shape}")

# 2. 重新计算jacobian（用torch.autograd.functional.jacobian函数）
J = F.jacobian(vgg5, inputs)
# print(f"J2.shape: {J.shape}, J2.prod: {torch.prod(torch.tensor(list(J.shape)))}")
J = J.reshape(J.shape[0],outputs.numel(),inputs.numel())
print(f"J2.shape: {J.shape}, J2.prod: {torch.prod(torch.tensor(list(J.shape)))}")

# 计算eta 源论文
# J = model.influence_jacobian(train_data)[:, :, :-1] / args.sigma  # 计算FIL（梯度）jacobian
# etas = J.pow(2).sum(1).mean(1).sqrt() # 计算dFIL(这时候不是spectral norm了) 

# 计算eta：drj摸索：
I = torch.matmul(J[0].t(), J[0])
dFIL = I.trace().div(inputs.numel())
eta = dFIL.sqrt()
print(f"eta: {eta}")


In [ ]:
# dFIL的两个要求: 可导 + unbiased
# x = torch.rand_like(torch.Tensor([1,5]))
x = torch.Tensor([0,0])
x.requires_grad_(True)
print(x.grad)
y = torch.nn.ReLU()
z = y(x).sum()
# z = torch.autograd.functional.jacobian(y, x)
z.backward()
print(x)
print(y)
print(z)
print(x.grad)

# 4. 现成函数调用

In [6]:
# 我实现的：
# dFIL inverse指标计算

eta_same_layer_list = []
eta_diff_layer_list=[]

metric = dFILInverseMetric()
# 对traingloader遍历计算所有 inverse dFIL
# for j, data in enumerate(tqdm.tqdm(testloader)):
for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
    # if j < 31705:
        # continue
    inputs, labels = data
    inputs, labels = inputs.to(args['device']), labels.to(args['device'])
    
    # inference
    outputs = client_net(inputs)

    eta = metric.quantify(model=client_net, inputs=inputs, outputs=outputs, with_outputs=True)
    # 打印
    # print(str(j)+": "+str(eta.item()))
    eta_same_layer_list.append(eta)
eta_diff_layer_list.append(eta_same_layer_list)

# 结果储存到csv中
matrix = np.array(eta_diff_layer_list) # 有点大
transpose = matrix.T # 一行一条数据，一列代表一个layer 
pd.DataFrame(data=transpose, columns=[split_layer]).to_csv(results_dir + f'inv_dFIL.csv',index=False)


100%|██████████| 1/1 [00:10<00:00, 10.71s/it]


In [25]:
# FIL 计算函数
import torch.autograd.functional as F
import torch
import time

# nips23
from torch.autograd.functional import jvp
import random
import math

def calc_tr(net, x, device, subsample=-1, jvp_parallelism=1): # nips'23 源码
    '''
    calc_tr 函数利用雅可比向量积（JVP）来估计网络对于输入数据的迹，
    这在分析网络的灵敏度或稳定性时非常有用。
    此外，通过支持子采样和并行处理，该函数还提供了一种在保持计算效率的同时估计迹的方法。
    '''
    print(f'x.shape: {x.shape}')
    
    # 定义一个局部函数 jvp_func**：这个函数接受两个参数 x 和 tgt，并返回 net.forward_first 方法的雅可比向量积（JVP）。
    # 这意味着 jvp_func 用于计算网络对于输入 x 在方向 tgt 上的一阶导数
    # tgt 计算雅各比向量积的向量
    def jvp_func(x, tgt):
        # return jvp(net.forward_first, (x,), (tgt,)) #返回 outputs, jacobian product
        return jvp(net.forward, (x,), (tgt,)) #返回 outputs, jacobian product

    # 获取一个batch中第一个数据的维度？d代表的是批次中第一个数据点展平后的特征数量，即输入数据的维度。
    d = x[0].flatten().shape[0] # 把一个batch的x展平，获取input dim

    # 用于存储每个输入数据点的迹
    tr = torch.zeros(x.shape[0], dtype=x.dtype).to(device)
    #print(f'd: {d}, {x.shape}')

    # 加速，矩阵降维，但是这个损伤精度，或许改成特征提取更好点？
    # Randomly subsample pixels for faster execution
    if subsample > 0:
        samples = random.sample(range(d), min(d, subsample))
    else:
        samples = range(d)

    #print(x.shape, d, samples)
    # jvp parallelism是数据并行的粒度？
    # 函数通过分批处理样本来计算迹，每批处理 jvp_parallelism 个样本
    for j in range(math.ceil(len(samples) / jvp_parallelism)): # 对于每个数据块
        tgts = []
        # 遍历每个数据块中的每个维度
        for k in samples[j*jvp_parallelism:(j+1)*jvp_parallelism]: # 提取整个batch中每个数据的特定维度
            tgt = torch.zeros_like(x).reshape(x.shape[0], -1) # 按照batch 排列？# 雅各比向量积的
            # 除了当前样本索引 k 对应的元素设置为 1。这相当于在计算迹时，每次只关注一个特征维度。
            tgt[:, k] = 1. # 提取tgt所有的样本的k的特征 计算雅各比向量积的向量，可用于计算trace
            tgt = tgt.reshape(x.shape) # 又变回x的形状
            tgts.append(tgt)
        tgts = torch.stack(tgts)

        def helper(tgt):
            batch_size = x.shape[0]
            vals_list = []
            grads_list = []
            for i in range(batch_size):
                val, grad = jvp_func(x[i], tgt[i])  # 对每个批次元素调用jvp_func
                vals_list.append(val)
                grads_list.append(grad)
            # 将结果列表转换为张量
            vals = torch.stack(vals_list)
            grad = torch.stack(grads_list)


            # vals, grad = vmap(jvp_func, randomness='same')(x, tgt)
            #print('grad shape: ', grad.shape)
            return torch.sum(grad * grad, dim=tuple(range(1, len(grad.shape)))), vals # 先求迹再求平方

        # vmap被替换
        trs,vals = [],[]
        for item in tgts:
            trs_, vals_ = helper(item)
            trs.append(trs_)
            vals.append(vals_)
        trs,vals = torch.stack(trs),torch.stack(vals)

        # trs, vals = vmap(helper, randomness='same')(tgts) # randomness for randomness control of dropout
        
        # vals are stacked results that are repeated by d (should be all the same)


        tr += trs.sum(dim=0)

    # Scale if subsampled
    if subsample > 0:
        tr *= d / len(samples)

    tr = tr/(d*1.0)
    tr = 1.0/tr

    print('tr: ',tr.shape, tr)
    return tr.cpu().item(), vals[0].squeeze(1)  # squeeze removes one dimension jvp puts

In [5]:
# maeng fisher

eta_same_layer_list = []
eta_diff_layer_list=[]

metric_trace = dFILInverseMetric()
for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
    # if j < 31705:
        # continue
    inputs, labels = data
    inputs, labels = inputs.to(args['device']), labels.to(args['device'])
    
    # inference
    # outputs = client_net(inputs)

    inputs = inputs.unsqueeze(0)
    eta,val = metric_trace.calc_tr(net=client_net, x=inputs, device=args['device'])
    # 打印
    # print(str(j)+": "+str(eta.item()))
    eta_same_layer_list.append(eta)
eta_diff_layer_list.append(eta_same_layer_list)

# 结果储存到csv中
matrix = np.array(eta_diff_layer_list) # 有点大
print("matrix: ",matrix)
transpose = matrix.T # 一行一条数据，一列代表一个layer 
pd.DataFrame(data=transpose, columns=[split_layer]).to_csv(results_dir + f'inv_dFIL_maeng.csv',index=False)



  0%|          | 0/1 [00:00<?, ?it/s]

x.shape: torch.Size([1, 2, 3, 32, 32])


100%|██████████| 1/1 [00:39<00:00, 39.37s/it]

matrix:  [[0.00075323]]


# 5. effective information

In [4]:
# effective fisher 计算函数
import torch.autograd.functional as F
import torch
import time

# nips23
from torch.autograd.functional import jvp
import random
import math

import pandas as pd

def computing_det_with_outputs(model, inputs, outputs, sigmas): # sigma_square
        # batchsize:
        batch_size = inputs.shape[0] # 一个batch的样本数目
        output_size = outputs[0].numel() # 一个样本的outputs长度
        input_size = inputs[0].numel() # 一个样本的outputs长度
        effect_fisher_sum = 0.0

        # 遍历单个样本: 换数据
        for i in range(batch_size):
            input_i = inputs[i].unsqueeze(0)

            # 计算jacobian
            J = F.jacobian(model, input_i)
            # J = J.reshape(J.shape[0],outputs.numel(),inputs.numel()) # (batch, out_size, in_size)
            J = J.reshape(output_size, input_size) # (batch, out_size, in_size)
            # print(f"J2.shape: {J.shape}, J2.prod: {torch.prod(torch.tensor(list(J.shape)))}")
            # 计算eta
            JtJ = torch.matmul(J.t(), J)
            I = 1.0/(sigmas)*JtJ
            # ddFIL  = I.trace().div(input_size*input_size)

            # 储存I
            # I_np = I.cpu().detach().numpy()
            # df = pd.DataFrame(I_np)
            # df.to_csv(f'{i}.csv',index=False,header=False)

            # print("I: ", I)
            # w = torch.det(I)
            # print('det I: ', I.det().log())

            f1 = input_size * torch.log(2*torch.pi*torch.exp(torch.tensor(1.0)))
            f2 = torch.logdet(I)
            # print('log det I: ',f2 )
            print('f1: ',f1)
            print('f2: ',f2)
            effect_fisher = 0.5 * (f1 - f2)
            effect_fisher_sum+=effect_fisher

            print("effect_fisher: ",effect_fisher)

        # print("Jt*J: ", JtJ)
        # print("Jt*J: ", JtJ.shape, JtJ)
        # print("I.shape: ", I.shape)
        # eta = dFIL
        # print(f"eta: {eta}")
        # print('t2-t1=',t2-t1, 't3-t2', t3-t2)
        effect_fisher_mean = effect_fisher_sum / batch_size
        return effect_fisher_mean.cpu().detach().numpy()

In [24]:
# effect fisher 指标计算

effectFisher_same_layer_list = []
Fishermetric = dFILInverseMetric() 
# for j, data in enumerate(tqdm.tqdm(testloader)): # 对testloader遍历
for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
    images, labels = data
    print('labels: ',labels)
    images, labels = images.to(args['device']), labels.to(args['device'])
    with torch.no_grad():
        # inference
        outputs = client_net(images).clone().detach()
        # fisher
        outputs = client_net(images)
        # images = images.unsqueeze(0)
        # effectFisher = Fishermetric._computing_det_with_outputs(model=client_net, inputs=images, outputs=outputs,sigmas = 0.01)
        effectFisher = computing_det_with_outputs(model=client_net, inputs=images, outputs=outputs,sigmas = 0.01)

        effectFisher_same_layer_list.append(effectFisher)
# print(f"Layer {split_layer} effecInfo: {sum(effecInfo_same_la yer_list)/len(effecInfo_same_layer_list)}")
print(f" effectfisher: {sum(effectFisher_same_layer_list)/len(effectFisher_same_layer_list)}")


  0%|          | 0/1 [00:00<?, ?it/s]

labels:  tensor([1, 0, 2])
f1:  tensor(11.3515)
f2:  tensor(28.3999, device='cuda:1')
effect_fisher:  tensor(-8.5242, device='cuda:1')


100%|██████████| 1/1 [00:00<00:00,  1.92it/s]

f1:  tensor(11.3515)
f2:  tensor(28.3999, device='cuda:1')
effect_fisher:  tensor(-8.5242, device='cuda:1')
f1:  tensor(11.3515)
f2:  tensor(28.3999, device='cuda:1')
effect_fisher:  tensor(-8.5242, device='cuda:1')
 effectfisher: -8.524190902709961


In [25]:
# effect entropy 计算函数
import math
import numpy as np
def shannon_entropy_pyent(time_series):
    """Calculate Shannon Entropy of the sample data.

    Parameters
    ----------
    time_series: np.ndarray | list[str]

    Returns
    -------
    ent: float
        The Shannon Entropy as float value
    """

    # Calculate frequency counts
    _, counts = np.unique(time_series, return_counts=True)
    total_count = len(time_series)
    # print('counts: ', counts)
    # print("total_count: ",total_count)

    # Calculate frequencies and Shannon entropy
    frequencies = counts / total_count
    # print("freq: ",frequencies)
    ent = -np.sum(frequencies * np.log(frequencies))

    return ent

import infotopo

def shannon_entropy_infotopo(x):
    def _entropy(x,base='e'): # 此时x应该是一个概率分布（离散的)
        x.flatten()
        if base=='e': 
            y = x * np.log(x+1e-9) # y<=0 # log是以e为底的
        else:
            y = x * self._lnyx(x+1e-9,base=base)
        # print("x.shape: ",x.shape)
        # print("y.shape: ",y.shape)
        return -np.sum(y) # 熵

    information_top = infotopo.infotopo(dimension_max = x.shape[1],
                                        dimension_tot = x.shape[1],
                                        sample_size = x.shape[0],
                                        nb_of_values = 9, # 不是很懂这个意思，为什么iris对应9？
                                        )
    # 计算联合分布的概率？（全排列）
    joint_prob = information_top._compute_probability(x)
    print('joint_prob: ',joint_prob)
    # 计算联合熵（全排列的）
    joint_prob_ent = information_top.simplicial_entropies_decomposition(x) # log2
    new_joint_prob_ent = {key: value * np.log(2) for key, value in joint_prob_ent.items()} #ln 转2为底 成 e为底
    print("joint_entropy: ",new_joint_prob_ent)
    # ent = information_top._compute_forward_entropies(x)
    # information_top.entropy_simplicial_lanscape(joint_prob) # 画图
    ent = _entropy(np.array(list(new_joint_prob_ent.values())))
    return ent


In [26]:
from pyentrp import entropy as ent

# effective entorpy
for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
# for j, data in enumerate(tqdm.tqdm(testloader)): 
    images, labels = data
    images, labels = images.to(args['device']), labels.to(args['device'])
    # print(images)
    with torch.no_grad():
        # ITE
        # effectEntro = Shannon_quantity(images)
        # print("effectEntro_ite: ",effectEntro)

        # PyEntropy
        effectEntro_pyent = 0.0
        for i in range(len(images[0])): # 对每个维度
            effectEntro_pyent  += shannon_entropy_pyent(images[:,i].flatten().detach().cpu().numpy())
            # print('effectEntro_pyent: ',effectEntro_pyent)
        # effectEntro_pyent = shannon_entropy(images.flatten(start_dim=1).detach().cpu().numpy())
        print('effectEntro_pyent: ',effectEntro_pyent)

        # infotopo
        effectEntro_infotopo = shannon_entropy_infotopo(images.flatten(start_dim=1).detach().cpu().numpy())
        # print('effectEntro_infotopo: ',effectEntro_infotopo)

100%|██████████| 1/1 [00:00<00:00,  6.73it/s]

effectEntro_pyent:  4.394449154672438
joint_prob:  {'0000': 0.3333333333333333, '0100': 0.3333333333333333, '0011': 0.3333333333333333}
Percent of tuples processed : 0
joint_entropy:  {(4,): 1.0986122886681096, (3,): 1.0986122886681096, (2,): 1.0986122886681096, (1,): 1.0986122886681096, (3, 4): 1.0986122886681096, (2, 4): 1.0986122886681096, (1, 4): 1.0986122886681096, (2, 3): 1.0986122886681096, (1, 3): 1.0986122886681096, (1, 2): 1.0986122886681096, (2, 3, 4): 1.0986122886681096, (1, 3, 4): 1.0986122886681096, (1, 2, 4): 1.0986122886681096, (1, 2, 3): 1.0986122886681096, (1, 2, 3, 4): 1.0986122886681096}


In [10]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
scaler = MinMaxScaler(feature_range=(0, 1))
# x = next(iter(one_data_loader))
for i,_ in one_data_loader:
        # print(i)
        y = scaler.fit_transform(i)
# print(y)

In [ ]:
y = torch.rand(10,341) # 最大能接受341个特征 
z = torch.randn(100,200)
e1 = Shannon_quantity(y)
e2 = Shannon_quantity(z)
print(e1)
print(e2)
# print(np.log(0))

170.22239461082756
335.1999980582977


In [9]:
import pandas as pd
dataPath = '/home/dengruijun/data/FinTech/DATASET/kaggle-dataset/bank/bank-additional-full.csv'

[X_train, y_train], [X_test, y_test] = preprocess_bank_dataset(dataPath)
df = pd.DataFrame(X_test)
df.describe()

===============processing data===============
X_train.shape: (32950, 63)
X_test.shape: (8238, 63)
y_train.shape: (32950, 1)
y_test.shape: (8238, 1) <class 'numpy.ndarray'>
===============processing data end===============


,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
count,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,...,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000,8238.000000
mean,0.289148,0.157198,0.023671,0.021850,0.063608,0.088614,0.032168,0.078174,0.064093,0.141539,...,0.284123,0.054413,0.020963,0.838213,0.081903,0.251712,0.335361,0.453437,0.089942,0.344142
std,0.453394,0.364010,0.152031,0.146202,0.244068,0.284203,0.176457,0.268462,0.244934,0.348598,...,0.171524,0.051057,0.030769,0.366935,0.121449,0.146193,0.252907,0.286649,0.055046,0.179637
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.160494,0.022977,0.000000,1.000000,0.000000,0.104167,0.199532,0.192469,0.036953,0.203781
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.234568,0.040057,0.018182,1.000000,0.000000,0.333333,0.269680,0.418410,0.093403,0.425709
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.382716,0.068524,0.036364,1.000000,0.142857,0.333333,0.296960,0.719665,0.143278,0.512287
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.769622,0.272727,1.000000,1.000000,0.479167,1.000000,1.000000,0.150759,0.512287


In [20]:
# effect Information 指标计算

effecInfo_diff_layer_list = []
effecInfo_same_layer_list = []
EntropyMetric = ULossMetric()
Fishermetric = dFILInverseMetric()

# for j, data in enumerate(tqdm.tqdm(testloader)): # 对testloader遍历
for j, data in enumerate(tqdm.tqdm(one_data_loader)): # 测试第一个testloader
    images, labels = data
    images, labels = images.to(args['device']), labels.to(args['device'])
    with torch.no_grad():
        # inference
        outputs = client_net(images).clone().detach()
        # entropy 
        effecEntro= EntropyMetric._entropy_prob_batch(images) # H(x)
        print("effecEntro: ",effecEntro)

        # fisher
        outputs = client_net(images)
        # effectFisher = Fishermetric._computing_det_with_outputs(model=client_net, inputs=images, outputs=outputs,sigmas = 0.01)
        effectFisher = computing_det_with_outputs(model=client_net, inputs=images, outputs=outputs,sigmas = 0.01)

        # 存储
        effecInfo_same_layer_list.append(effecEntro-effectFisher)
        
print(f"Layer {split_layer} effecInfo: {sum(effecInfo_same_layer_list)/len(effecInfo_same_layer_list)}")
effecInfo_diff_layer_list.append(effecInfo_same_layer_list)

# 保存到csv中
matrix = np.array(effecInfo_diff_layer_list) # 有点大，x
transpose = matrix.T # 一行一条数据，一列代表一个layer 
# pd.DataFrame(data=transpose, columns=[i for i in split_layer_list]).to_csv(results_dir + f'effecInfo-bs{batch_size}.csv',index=False)
pd.DataFrame(data=transpose, columns=[split_layer]).to_csv(results_dir + f'effecInfo.csv',index=False)


  0%|          | 0/1 [00:00<?, ?it/s]

effecEntro:  0.587471032886238
I:  tensor([[ 7.5991,  0.0214, -0.1178,  ...,  0.1394, -0.0462,  0.2453],
        [ 0.0214,  9.0257, -0.1333,  ..., -0.0265, -0.7587, -0.6064],
        [-0.1178, -0.1333,  7.9887,  ...,  0.1769,  0.4307,  0.1134],
        ...,
        [ 0.1394, -0.0265,  0.1769,  ...,  7.8924,  0.1008, -0.3792],
        [-0.0462, -0.7587,  0.4307,  ...,  0.1008,  8.1875, -0.1230],
        [ 0.2453, -0.6064,  0.1134,  ..., -0.3792, -0.1230,  7.2750]],
       device='cuda:1')
det I:  tensor(-inf, device='cuda:1')
log det I:  tensor(-575.6401, device='cuda:1')
effect_fisher:  tensor(1139.1831, device='cuda:1')


100%|██████████| 1/1 [00:02<00:00,  2.01s/it]

I:  tensor([[ 6.5576e+00,  7.9808e-02, -1.4507e-01,  ...,  3.2295e-02,
          4.8863e-02,  1.3413e-01],
        [ 7.9808e-02,  7.9728e+00, -1.3367e-01,  ..., -1.0713e-01,
         -6.0581e-01, -5.0328e-01],
        [-1.4507e-01, -1.3367e-01,  7.2373e+00,  ...,  2.4361e-01,
          2.0677e-01,  3.4368e-03],
        ...,
        [ 3.2295e-02, -1.0713e-01,  2.4361e-01,  ...,  7.0409e+00,
          1.0706e-01, -3.2667e-01],
        [ 4.8863e-02, -6.0581e-01,  2.0677e-01,  ...,  1.0706e-01,
          7.1875e+00, -1.9418e-01],
        [ 1.3413e-01, -5.0328e-01,  3.4368e-03,  ..., -3.2667e-01,
         -1.9418e-01,  6.4056e+00]], device='cuda:1')
det I:  tensor(-inf, device='cuda:1')
log det I:  tensor(-674.2421, device='cuda:1')
effect_fisher:  tensor(1188.4841, device='cuda:1')
Layer 3 effecInfo: -1163.2461471311763
